In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt



# Device Configuration

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#Data

In [ ]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import random_split

normalize = transforms.Normalize(
    mean=[0.4914, 0.4822, 0.4465],
    std=[0.2023, 0.1994, 0.2010]
)

resize = transforms.Resize((227, 227))
transform = transforms.Compose([resize,transforms.ToTensor(),normalize] )

train_data = datasets.CIFAR10(
    root='./data/',
    train=True,
    download=True,
    transform=transform
)
test_data=datasets.CIFAR10(
    root='./data/',
    train=False,
    download=True,
    transform=transform
)
train_set, val_set = random_split( train_data,[40000, 10000])

train_loader=torch.utils.data.DataLoader(train_set,batch_size=32,shuffle=True)
validation_loader=torch.utils.data.DataLoader(val_set,batch_size=32,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


# Model

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, padding=2),
             nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, padding=1),
             nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),
             nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
             nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        prob = F.softmax(x, dim=1)
        return x


model=AlexNet(10).to(device)

# Defining  Training Loop

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn):
    for epoch in range(n_epochs):

      train_loss=0.0
      model.train()
      for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

      val_loss=0.0
      model.eval()
      for inputs, labels in validation_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        val_loss += loss.item()

      print("Epoch: {} /{} Train Loss: {} Val Loss: {}".format(epoch+1,n_epochs, train_loss/len(train_loader), val_loss/len(validation_loader)))



# Training


In [ ]:
optimizer=optim.SGD(model.parameters(),lr=0.005)
loss_fn=nn.CrossEntropyLoss()
training_loop(
    n_epochs=10,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn)



Epoch: 1 /10 Train Loss: 1.5093601583003997 Val Loss: 1.2020336857999856
Epoch: 2 /10 Train Loss: 1.101154917907715 Val Loss: 0.8943101211477773
Epoch: 3 /10 Train Loss: 0.9045708149909973 Val Loss: 0.8160366955847024
Epoch: 4 /10 Train Loss: 0.7941016122102738 Val Loss: 1.0441853007950341
Epoch: 5 /10 Train Loss: 0.7163511782884597 Val Loss: 0.7277457853094839
Epoch: 6 /10 Train Loss: 0.6448807433128357 Val Loss: 0.6465621618226695
Epoch: 7 /10 Train Loss: 0.5894307572245597 Val Loss: 0.6466705874798778
Epoch: 8 /10 Train Loss: 0.5433457347512245 Val Loss: 0.6347414367496015
Epoch: 9 /10 Train Loss: 0.5008763216018677 Val Loss: 0.634696934550715
Epoch: 10 /10 Train Loss: 0.4658649053752422 Val Loss: 0.5510636040101797


In [ ]:
num_correct = 0.0
for x_test_batch, y_test_batch in test_loader:
  model.eval()
  y_test_batch = y_test_batch.to(device)
  x_test_batch = x_test_batch.to(device)
  y_pred_batch = model(x_test_batch)
  _, predicted = torch.max(y_pred_batch, 1)
  num_correct += (predicted ==y_test_batch).float().sum()
  accuracy = num_correct/(len(test_loader) *test_loader.batch_size)
print(len(test_loader), test_loader.batch_size)
print("Test Accuracy: {}".format(accuracy))

313 32
Test Accuracy: 0.809105396270752
